In [1]:
using Pkg
Pkg.update()

   Updating registry at `~/.julia/registries/General`
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [2]:
pkgList = ["DataFramesMeta","PackageCompiler","CSV","Flux","HTTP","JSON","DataFrames","Plots","TimeSeries","MultivariateStats"]
for pkg in pkgList
    Pkg.add(pkg)
    
end

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/

In [3]:
ENV["PYTHON"] = "/Users/kahingleung/PycharmProjects/mylightning/venv/bin/python3.8"
Pkg.build("PyCall")

   Building Conda ─→ `~/.julia/packages/Conda/x5ml4/deps/build.log`
   Building PyCall → `~/.julia/packages/PyCall/tqyST/deps/build.log`


In [4]:
using Flux
using Flux: @epochs
using HTTP
using JSON
using Pkg
using DataFrames,DataFramesMeta
using Dates
using Statistics
using Statistics: mean
using Printf
using Plots
using Base
using CSV
using Hyperopt
using TimeSeries
using MultivariateStats

In [5]:
y0 = Dates.format(today(), "yyyy-mm-dd")
y1 = Dates.format(today()-Year(1), "yyyy-mm-dd")
y2 = Dates.format(today()-Year(2), "yyyy-mm-dd")
y3 = Dates.format(today()-Year(3), "yyyy-mm-dd")

"2018-01-17"

In [6]:
function getFX(fromYear,toYear)
    #url_query = "https://api.exchangeratesapi.io/history?start_at="*fromYear*"&end_at="*toYear*"&symbols=PHP,THB,IDR,GBP&base=USD"
    url_query = "https://api.exchangeratesapi.io/history?start_at="*fromYear*"&end_at="*toYear*"&base=USD"
    raw_response = HTTP.request("GET", url_query)
    rsp = JSON.parse(String(raw_response.body))
    rates = get(rsp,"rates",0)
    #df1 = DataFrame(date=String[],PHP=Float64[],THB=Float64[],IDR=Float64[],GBP=Float64[])
    df = DataFrame(date=String[], CUR=String[], rate=Float32[])
    for (k,v) in rates
        #print(k,get(v,"EUR",0) ,get(v,"JPY",0))
        #d = Dict("date" => k, "PHP" => get(v,"PHP",0), "THB" => get(v,"THB",0), "IDR" => get(v,"IDR",0), "GBP" => get(v,"GBP",0))
        for cur in keys(v)
            r = get(v,cur,0)
            row = DataFrame(date = [k], CUR=[cur], rate=[r])
            df = vcat(df,row)
        end
    end
    transform!(df, :date => ( x -> Date.(x, Dates.DateFormat("yyyy-mm-dd")) ) => :ondate)
    return df
end
function getBTC(from,to)
    headers = ["apikey" => "101c5169-770f-4add-8be0-6131419b7208"]
    url_query = "http://api.bitdataset.com/v1/ohlcv/history/BITFINEX:BTCUSD?period=D1&start="*from*"&end="*to*"&limit=1000"
    raw_response = HTTP.request("GET", url_query, headers)
    rsp = JSON.parse(String(raw_response.body))
    df2 = DataFrame(high = Float32[],low = Float32[],open = Float32[],close = Float32[],volume = Float32[], time = String[])
    for d in rsp
        push!(df2,d)
    end
    transform!(df2, :time => ( x -> split.(x, "T") ) => :dateary)
    transform!(df2, :dateary => ( x -> getindex.(x,1 ) ) => :datestr)
    transform!(df2, :datestr => ( x -> Date.(x, Dates.DateFormat("yyyy-mm-dd")) ) => :ondate)
    return df2
end
function loadData(from,to)
    d2 = getFX(from,to)
    d1 = getBTC(from,to)
    
    df3 = innerjoin(select(d1,[:open,:close,:high,:low,:volume,:ondate]), d2, on = :ondate)
    return df3
end



loadData (generic function with 1 method)

In [7]:
infile="/Users/kahingleung/Downloads/gold-julia/btc.csv"
if isfile(infile)
    rawbtc = CSV.File(infile)
    rawbtc = DataFrame(rawbtc)
    for c in ["high","low","open","close","volume"]
        transform!(rawbtc,c => (x -> Float32.(x) )=> c)
    end
else
    btc1 = getBTC(y1,y0)
    btc2 = getBTC(y2,y1)
    btc3 = getBTC(y3,y2)
    rawbtc = vcat(btc1,btc2,btc3)
    CSV.write(infile, rawbtc)
end
rawbtc

,high,low,open,close,volume,time,dateary
,Float32,Float32,Float32,Float32,Float32,String,String
1,8839.9,8579.8,8807.7,8720.6,3.72522,2020-01-16T00:00:00.000Z,"SubString{String}[""2020-01-16"", ""00:00:00.000Z""]"
2,9000.0,8674.2,8719.55,8875.4,3.98645,2020-01-17T00:00:00.000Z,"SubString{String}[""2020-01-17"", ""00:00:00.000Z""]"
3,8965.0,8791.12,8875.5,8891.6,3.53917,2020-01-18T00:00:00.000Z,"SubString{String}[""2020-01-18"", ""00:00:00.000Z""]"
4,9169.5,8450.0,8891.5,8698.7,4.09368,2020-01-19T00:00:00.000Z,"SubString{String}[""2020-01-19"", ""00:00:00.000Z""]"
5,8741.1,8519.6,8698.7,8638.2,3.61333,2020-01-20T00:00:00.000Z,"SubString{String}[""2020-01-20"", ""00:00:00.000Z""]"
6,8787.17,8500.0,8638.2,8734.1,3.51204,2020-01-21T00:00:00.000Z,"SubString{String}[""2020-01-21"", ""00:00:00.000Z""]"
7,8799.9,8580.5,8734.2,8663.9,3.36957,2020-01-22T00:00:00.000Z,"SubString{String}[""2020-01-22"", ""00:00:00.000Z""]"
8,8670.4,8296.0,8663.8,8403.57,3.75617,2020-01-23T00:00:00.000Z,"SubString{String}[""2020-01-23"", ""00:00:00.000Z""]"
9,8523.0,8246.0,8403.57,8447.13,3.54449,2020-01-24T00:00:00.000Z,"SubString{String}[""2020-01-24"", ""00:00:00.000Z""]"


In [8]:
infile="/Users/kahingleung/Downloads/gold-julia/fx.csv"
if isfile(infile)
    rawdata = CSV.File(infile)
    rawdata = DataFrame(rawdata)
    for c in ["rate"]
        transform!(rawdata,c => (x -> Float32.(x) )=> c)
    end
else
    df1 = getFX(y1,y0)
    df2 = getFX(y2,y1)
    df3 = getFX(y3,y2)
    rawdata = vcat(df1,df2,df3)
    CSV.write(infile, rawdata)
end
rawdata

,date,CUR,rate,ondate
,Date,String,Float32,Date
1,2020-03-18,CZK,24.8345,2020-03-18
2,2020-03-18,PLN,4.11652,2020-03-18
3,2020-03-18,MXN,23.9632,2020-03-18
4,2020-03-18,TRY,6.44695,2020-03-18
5,2020-03-18,ISK,139.656,2020-03-18
6,2020-03-18,USD,1.0,2020-03-18
7,2020-03-18,RUB,79.656,2020-03-18
8,2020-03-18,EUR,0.914578,2020-03-18
9,2020-03-18,GBP,0.84315,2020-03-18


In [9]:
rawdata = select(rawdata,[:ondate,:CUR,:rate])
rawdata = combine(groupby(rawdata, [:ondate,:CUR]),:rate=>mean)

,ondate,CUR,rate_mean
,Date,String,Float32
1,2020-03-18,CZK,24.8345
2,2020-03-18,PLN,4.11652
3,2020-03-18,MXN,23.9632
4,2020-03-18,TRY,6.44695
5,2020-03-18,ISK,139.656
6,2020-03-18,USD,1.0
7,2020-03-18,RUB,79.656
8,2020-03-18,EUR,0.914578
9,2020-03-18,GBP,0.84315


In [10]:
df = DataFrame()
df.ondate = unique(rawdata.ondate)
datecol = df
for c in unique(rawdata.CUR)
    df1 = @linq rawdata |> where(:CUR .== c) |> select(:ondate,:rate_mean)
    coldata = leftjoin(datecol,df1,on=:ondate)
    df[:,c] = coldata[:,:rate_mean]
end
df = sort(df,order(:ondate))

,ondate,CZK,PLN,MXN,TRY,ISK,USD,RUB
,Date,Float32?,Float32?,Float32?,Float32?,Float32?,Float32?,Float32?
1,2018-01-16,20.8635,3.41071,18.7725,3.79697,missing,1.0,56.4392
2,2018-01-17,20.8531,3.41826,18.7975,3.82316,missing,1.0,56.8597
3,2018-01-18,20.7315,3.40539,18.6493,3.79027,missing,1.0,56.5694
4,2018-01-19,20.7532,3.404,18.6175,3.79902,missing,1.0,56.5748
5,2018-01-22,20.7525,3.40771,18.634,3.80358,missing,1.0,56.6075
6,2018-01-23,20.7388,3.40403,18.8278,3.77949,missing,1.0,56.495
7,2018-01-24,20.5424,3.36488,18.5432,3.74539,missing,1.0,56.3249
8,2018-01-25,20.457,3.34239,18.442,3.74063,missing,1.0,55.8407
9,2018-01-26,20.39,3.33081,18.5486,3.75,missing,1.0,55.9059


In [11]:
dropmissing!(df)

,ondate,CZK,PLN,MXN,TRY,ISK,USD,RUB,EUR
,Date,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32
1,2018-02-01,20.2801,3.33438,18.5657,3.74645,100.337,1.0,56.2822,0.802633
2,2018-02-02,20.1665,3.32541,18.4566,3.746,100.224,1.0,56.2711,0.800512
3,2018-02-05,20.254,3.34285,18.5888,3.75997,100.482,1.0,56.6708,0.803859
4,2018-02-06,20.4664,3.37773,18.8109,3.78352,101.387,1.0,57.3531,0.811096
5,2018-02-07,20.449,3.37129,18.6795,3.78351,101.313,1.0,57.1576,0.810504
6,2018-02-08,20.6064,3.40312,18.7674,3.8117,102.187,1.0,57.8565,0.816193
7,2018-02-09,20.6429,3.41424,18.8163,3.8271,102.013,1.0,58.2624,0.814797
8,2018-02-12,20.6785,3.40439,18.6222,3.80396,102.259,1.0,57.9671,0.815461
9,2018-02-13,20.5838,3.38774,18.6312,3.79997,101.678,1.0,57.7765,0.810833


In [12]:
df = innerjoin(select(rawbtc,[:open,:close,:high,:low,:volume,:ondate]), df, on = :ondate)

,open,close,high,low,volume,ondate,CZK,PLN,MXN
,Float32,Float32,Float32,Float32,Float32,Date,Float32,Float32,Float32
1,8807.7,8720.6,8839.9,8579.8,3.72522,2020-01-16,22.5356,3.78942,18.7932
2,8719.55,8875.4,9000.0,8674.2,3.98645,2020-01-17,22.6386,3.8141,18.7557
3,8698.7,8638.2,8741.1,8519.6,3.61333,2020-01-20,22.6658,3.82797,18.6685
4,8638.2,8734.1,8787.17,8500.0,3.51204,2020-01-21,22.5515,3.81808,18.7309
5,8734.2,8663.9,8799.9,8580.5,3.36957,2020-01-22,22.6696,3.82197,18.7064
6,8663.8,8403.57,8670.4,8296.0,3.75617,2020-01-23,22.6842,3.82644,18.7113
7,8403.57,8447.13,8523.0,8246.0,3.54449,2020-01-24,22.8002,3.85727,18.7739
8,8621.6,8912.0,9001.86,8585.0,3.79774,2020-01-27,22.8853,3.87438,18.9311
9,8912.06,9397.0,9414.0,8911.1,3.95999,2020-01-28,22.9178,3.88287,18.8383


In [13]:
[c for c in names(df) if c != "ondate"]

38-element Array{String,1}:
 "open"
 "close"
 "high"
 "low"
 "volume"
 "CZK"
 "PLN"
 "MXN"
 "TRY"
 "ISK"
 "USD"
 "RUB"
 "EUR"
 ⋮
 "MYR"
 "ZAR"
 "CAD"
 "PHP"
 "HKD"
 "HRK"
 "INR"
 "AUD"
 "NOK"
 "DKK"
 "RON"
 "ILS"

In [14]:
using PyCall
yf = pyimport("yfinance")
#sym = "gbtc"
sym = "2840.HK"
ticker = yf.Ticker("gbtc")
etf = ticker.history(period="3y")

function pd_to_df(df_pd)
    df= DataFrame()
    for col in df_pd.columns
        df[!, col] = getproperty(df_pd, col).values
    end
    df[!,:Date] = collect(df_pd[:index])
    return df
end
etf = pd_to_df(etf)

transform!(etf, :Date => ( x -> Date.(Dates.year.(x),Dates.month.(x),Dates.day.(x)) ) => :ondate)
for c in ["High","Low","Open","Close","Stock Splits"]
    transform!(etf,c => (x -> Float32.(x) )=> c)
end

In [15]:
etf = select(etf,["Close","ondate"])

,Close,ondate
,Float32,Date
1,19.0769,2018-01-16
2,17.978,2018-01-17
3,20.4066,2018-01-18
4,19.4289,2018-01-19
5,18.5275,2018-01-22
6,18.161,2018-01-23
7,18.3522,2018-01-24
8,18.5165,2018-01-25
9,18.0769,2018-01-26


In [16]:
df3 = innerjoin(df, etf, on = :ondate)
sort!(df3, [:ondate])


,open,close,high,low,volume,ondate,CZK,PLN,MXN
,Float32,Float32,Float32,Float32,Float32,Date,Float32,Float32,Float32
1,9136.0,8176.0,9439.6,7916.0,4.8968,2018-03-14,20.5732,3.39745,18.5775
2,8175.6,8250.1,8418.0,7665.1,4.9167,2018-03-15,20.5948,3.41318,18.6892
3,8249.5,8250.0,8604.5,7902.0,4.75132,2018-03-16,20.6609,3.42842,18.729
4,8200.3,8600.2,8717.0,8085.7,4.86481,2018-03-19,20.6613,3.42603,18.8375
5,8600.2,8899.7,9040.0,8305.1,4.73468,2018-03-20,20.7095,3.44387,18.7428
6,8899.8,8888.2,9177.5,8752.0,4.6322,2018-03-21,20.6918,3.44815,18.5479
7,8888.3,8699.3,9095.92,8450.0,4.73551,2018-03-22,20.6228,3.42782,18.4934
8,8696.0,8914.4,8917.6,8271.1,4.65265,2018-03-23,20.5775,3.42313,18.5326
9,8445.3,8131.12,8499.1,7835.9,4.75119,2018-03-26,20.5028,3.40827,18.4334


In [17]:
features = [c for c in names(df) if c != "ondate" && c != "Close"]

38-element Array{String,1}:
 "open"
 "close"
 "high"
 "low"
 "volume"
 "CZK"
 "PLN"
 "MXN"
 "TRY"
 "ISK"
 "USD"
 "RUB"
 "EUR"
 ⋮
 "MYR"
 "ZAR"
 "CAD"
 "PHP"
 "HKD"
 "HRK"
 "INR"
 "AUD"
 "NOK"
 "DKK"
 "RON"
 "ILS"

In [18]:
ts = TimeArray(df3,timestamp=:ondate)

701×39 TimeArray{Float32,2,Date,Array{Float32,2}} 2018-03-14 to 2021-01-15
│            │ open      │ close     │ high      │ low       │ volume │
├────────────┼───────────┼───────────┼───────────┼───────────┼────────┤
│ 2018-03-14 │ 9136.0    │ 8176.0    │ 9439.6    │ 7916.0    │ 4.8968 │
│ 2018-03-15 │ 8175.6    │ 8250.1    │ 8418.0    │ 7665.1    │ 4.9167 │
│ 2018-03-16 │ 8249.5    │ 8250.0    │ 8604.5    │ 7902.0    │ 4.7513 │
│ 2018-03-19 │ 8200.3    │ 8600.2    │ 8717.0    │ 8085.7    │ 4.8648 │
│ 2018-03-20 │ 8600.2    │ 8899.7    │ 9040.0    │ 8305.1    │ 4.7347 │
│ 2018-03-21 │ 8899.8    │ 8888.2    │ 9177.5    │ 8752.0    │ 4.6322 │
│ 2018-03-22 │ 8888.3    │ 8699.3    │ 9095.922  │ 8450.0    │ 4.7355 │
│ 2018-03-23 │ 8696.0    │ 8914.4    │ 8917.6    │ 8271.1    │ 4.6526 │
│ 2018-03-26 │ 8445.3    │ 8131.123  │ 8499.1    │ 7835.9    │ 4.7512 │
│ 2018-03-27 │ 8133.2    │ 7784.6    │ 8210.0    │ 7716.6    │ 4.7024 │
│ 2018-03-28 │ 7785.0    │ 7936.0    │ 8120.2    │ 7702.1    

In [19]:
pct = percentchange(ts)
pct = moving(mean, pct, 5)

696×39 TimeArray{Float32,2,Date,Array{Float32,2}} 2018-03-21 to 2021-01-15
│            │ open    │ close   │ high    │ low     │ volume  │ CZK     │
├────────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│ 2018-03-21 │ -0.0037 │ 0.017   │ -0.0041 │ 0.0207  │ -0.0108 │ 0.0012  │
│ 2018-03-22 │ 0.0171  │ 0.0109  │ 0.0157  │ 0.0201  │ -0.0072 │ 0.0003  │
│ 2018-03-23 │ 0.0109  │ 0.0159  │ 0.0074  │ 0.0097  │ -0.0039 │ -0.0008 │
│ 2018-03-26 │ 0.0064  │ -0.0102 │ -0.0046 │ -0.0055 │ -0.0045 │ -0.0015 │
│ 2018-03-27 │ -0.0108 │ -0.0257 │ -0.0188 │ -0.0139 │ -0.0012 │ -0.0012 │
│ 2018-03-28 │ -0.0263 │ -0.0215 │ -0.0241 │ -0.0251 │ -0.0036 │ -0.0015 │
│ 2018-03-29 │ -0.0222 │ -0.0385 │ -0.0259 │ -0.0394 │ 0.0071  │ 0.0001  │
│ 2018-04-03 │ -0.0401 │ -0.0346 │ -0.0336 │ -0.0312 │ 0.0028  │ 0.0003  │
│ 2018-04-04 │ -0.0243 │ -0.0338 │ -0.0266 │ -0.0307 │ -0.0001 │ 0.0014  │
│ 2018-04-05 │ -0.0337 │ -0.0258 │ -0.0328 │ -0.0308 │ -0.0009 │ 0.0007  │
│ 2018-04-06 │ -0.0256 │ 

In [20]:
df = DataFrame(pct)

,timestamp,open,close,high,low,volume,CZK
,Date,Float32,Float32,Float32,Float32,Float32,Float32
1,2018-03-21,-0.00368886,0.0170065,-0.00414634,0.0206806,-0.010816,0.00115174
2,2018-03-22,0.0170772,0.0109433,0.0157209,0.0201183,-0.00716839,0.000274513
3,2018-03-23,0.0109423,0.015891,0.0073691,0.00970275,-0.00394061,-0.000806786
4,2018-03-26,0.00636936,-0.0101721,-0.00463188,-0.00547011,-0.004482,-0.00153688
5,2018-03-27,-0.0107751,-0.0256604,-0.0188456,-0.0139419,-0.00118463,-0.00122511
6,2018-03-28,-0.0263049,-0.0215122,-0.0240751,-0.0250798,-0.00355051,-0.00149559
7,2018-03-29,-0.0221646,-0.0384816,-0.025923,-0.0394479,0.00707485,0.00012954
8,2018-04-03,-0.0400828,-0.0346418,-0.0336238,-0.0312358,0.00279736,0.0003229
9,2018-04-04,-0.0243302,-0.0338044,-0.0265735,-0.0306811,-6.19597e-5,0.00135722


In [21]:
target = ["Close"]
#features = vcat(unique(rawdata.CUR) , ["open","close","high","low","volume"])
#features = ["AUD","GBP","open","close","high","low","volume"]
features = [c for c in names(df) if c != "timestamp" && c != "Close"]

38-element Array{String,1}:
 "open"
 "close"
 "high"
 "low"
 "volume"
 "CZK"
 "PLN"
 "MXN"
 "TRY"
 "ISK"
 "USD"
 "RUB"
 "EUR"
 ⋮
 "MYR"
 "ZAR"
 "CAD"
 "PHP"
 "HKD"
 "HRK"
 "INR"
 "AUD"
 "NOK"
 "DKK"
 "RON"
 "ILS"

In [22]:

for col in vcat(features, target)
    transform!(df,[col] => (x -> map(a->Float32(100*a),x)) => [col]) 
end


In [23]:
dropmissing!(df)
df = select(df,vcat(features, target))

,open,close,high,low,volume,CZK,PLN
,Float32,Float32,Float32,Float32,Float32,Float32,Float32
1,-0.368886,1.70065,-0.414634,2.06806,-1.0816,0.115174,0.296998
2,1.70772,1.09433,1.57209,2.01183,-0.716839,0.0274513,0.0864524
3,1.09423,1.5891,0.73691,0.970275,-0.394061,-0.0806786,-0.030212
4,0.636936,-1.01721,-0.463188,-0.547011,-0.4482,-0.153688,-0.10312
5,-1.07751,-2.56605,-1.88456,-1.39419,-0.118463,-0.122511,-0.200041
6,-2.63049,-2.15122,-2.40751,-2.50798,-0.355051,-0.149559,-0.319044
7,-2.21646,-3.84816,-2.5923,-3.94479,0.707485,0.012954,-0.0598449
8,-4.00828,-3.46418,-3.36238,-3.12358,0.279736,0.03229,-0.0204208
9,-2.43302,-3.38044,-2.65735,-3.06811,-0.00619597,0.135722,0.0937925


In [24]:
names(df)[end]

"Close"

In [25]:
function timeseriesDataset(df,seqlen,features,target)
    xtrain = Matrix[]
    ytrain = Float32[]
    #seqlen=7
    f = getindex(size(features),1)
    #println(f)
    len=getindex(size(df),1)
    mx = transpose(convert(Matrix,df[:,features]))
    #println(size(mx))
    M = MultivariateStats.fit(PCA, mx; maxoutdim=3)
    Yte = MultivariateStats.transform(M, mx)
    #println(size(Yte))
    #Yte = mx
    for i in 1:len-seqlen-1
        #mx = select(df[i:i+seqlen-1,:],features)
        #mx = reshape(convert(Matrix,mx),(f,seqlen))
        #print(size(mx))
        mx = Yte[:,i:i+seqlen-1]
        #println(size(mx))
        xtrain = vcat(xtrain,[mx])

        #my = df[i+1:i+seqlen,target]
        my = df[i+seqlen,target]
        #my = getindex(convert(Vector,my),1)
        #println(size(my))
        ytrain = vcat(ytrain,[my])
    end
    return (xtrain,ytrain)
end



timeseriesDataset (generic function with 1 method)

In [26]:
features = names(df)[1:end-2]
target = names(df)[end]
seqlen=7
(xtrain,ytrain) = timeseriesDataset(df,seqlen,features,target)

(Array{T,2} where T[Float32[-0.83390373 -2.5671592 … 5.416995 6.9080853; 1.7678396 1.0089825 … 0.5286435 -0.723088; -1.0758958 -0.20925191 … 0.92425704 0.595779], Float32[-2.5671592 -1.5877119 … 6.9080853 7.571924; 1.0089825 0.64681685 … -0.723088 0.007183075; -0.20925191 0.17409013 … 0.595779 0.38345098], Float32[-1.5877119 1.2591164 … 7.571924 6.4196854; 0.64681685 0.6532201 … 0.007183075 0.15390736; 0.17409013 1.1805279 … 0.38345098 -0.24775037], Float32[1.2591164 4.0280805 … 6.4196854 6.77892; 0.6532201 0.4556574 … 0.15390736 0.35419083; 1.1805279 1.0301441 … -0.24775037 0.006040454], Float32[4.0280805 5.416995 … 6.77892 6.906805; 0.4556574 0.5286435 … 0.35419083 -0.15500414; 1.0301441 0.92425704 … 0.006040454 -0.26763615], Float32[5.416995 6.9080853 … 6.906805 3.5098002; 0.5286435 -0.723088 … -0.15500414 0.43243992; 0.92425704 0.595779 … -0.26763615 -0.33431804], Float32[6.9080853 7.571924 … 3.5098002 3.077043; -0.723088 0.007183075 … 0.43243992 1.3694549; 0.595779 0.38345098 … -0

In [27]:
function build_model(Nh)
    a = floor(Int8,Nh)
    return Chain(
    x -> Flux.unsqueeze(Flux.unsqueeze(x,3),4),
    # First convolution, operating upon a 28x28 image
    Conv((2, 2), 1=>a, pad=(1,1), relu),
    MaxPool((2,2)),

    # Second convolution, operating upon a 14x14 image
    Conv((2, 2), a=>Nh, pad=(1,1), relu),
    MaxPool((2,2)),

    # Third convolution, operating upon a 7x7 image
    #Conv((2, 2), Nh=>Nh, pad=(1,1), relu),
    #MaxPool((2,2)),

    # Reshape 3d tensor into a 2d one using `Flux.flatten`, at this point it should be (3, 3, 32, N)
    Flux.flatten,
    Dropout(0.1),
    (x->transpose(x)),
    GRU(1,Nh),
    GRU(Nh,Nh),
    (x -> x[:,end]),
    Dense(Nh, 1),
    (x -> x[1]))
end

build_model (generic function with 1 method)

In [28]:
function build_lstm()
    #Nt = seqlen       # time steps
    Nin,Nout = 3,1 # input size, output size
    #Nh = 5        # hidden dim
    #num_layers = 2
    layers = []
    layers = vcat(layers,[LSTM(Nin,Nh),Dropout(dr)])
    for i in 1:num_layers-1
        layers = vcat(layers,[LSTM(Nh,Nh),Dropout(dr)])
    end
    #layers = vcat(layers,[Dropout(dr)])
    linear1 = Chain(Dense(Nh,10),Dense(10,1))
    #lstm = Chain(Dense(Nin,Nh),Dense(Nh,1))
    #lstm(x) = linear1(foldl((x, m) -> m(x), layers, init = x)[:,end])
    lstm = Chain(LSTM(Nin,Nh),Dropout(dr),Dense(Nh,10),Dense(10,Nout)) # simple lstm
    #lstm(x) = linear1(foldl((x, m) -> m(x), layers, init = x))
    return lstm
end

build_lstm (generic function with 1 method)

In [29]:
m = build_model(32)

Chain(#29, Conv((2, 2), 1=>32, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((2, 2), 32=>32, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), flatten, Dropout(0.1), #30, Recur(GRUCell(1, 32)), Recur(GRUCell(32, 32)), #31, Dense(32, 1), #32)

In [30]:
function test_mse_loss(x,y)
        yh = m.(x)
        #println(yh)
        #print(y)
        e = Flux.mae(yh,y)
        println(size(e))
        println(e)
        return e
    end
test_mse_loss(xtrain,ytrain)

()
1.8766643


1.8766643f0

In [31]:

function myTrain(df,seqlen,Nh,lr,num_epoch,i)

    (xtrain,ytrain) = timeseriesDataset(df,seqlen,features,target)
    #println(size(xtrain[1]))
    thd = 250
    xtest,ytest = xtrain[end-thd+1:end], ytrain[end-thd+1:end]
    xtrain,ytrain = xtrain[1:end-thd], ytrain[1:end-thd]
    batchsize = 20
    train_loader = Flux.Data.DataLoader(xtrain,ytrain, batchsize=batchsize,shuffle=false)
    m = build_model(Nh)
    function mse_loss(x,y)
        yh = m.(x)
        e = Flux.mae(yh,y)
        return e
    end
    evalcb() = @show mse_loss(xtest,ytest)

    throttled_cb = @show mse_loss(xtest,ytest)
    
    
    @epochs num_epoch Flux.train!(mse_loss,Flux.params(m),train_loader,RMSProp(lr))
    
    prediction = m.(xtest)
    println(size(prediction))
    finalLoss = mse_loss(xtest,ytest)
    p = map(x -> x[end],prediction)
    y = map(x -> x[end],ytest)
    lp = getindex(size(prediction),1)
    ly = getindex(size(y),1)
    plot(1:lp,p,size = (1000, 700),color="red",tickfontsize=28,guidefontsize=28,legendfontsize=28)
    plot!(1:ly,y,size = (1000, 700),color="blue",tickfontsize=28,guidefontsize=28,legendfontsize=28)
    dir="/Users/kahingleung/Downloads/gold-julia/"
    png("/Users/kahingleung/Downloads/gold-julia/plot-"*string(i)*".png")
    return (m,ytest,prediction,finalLoss)
end

function myObjective(df,seqlen,Nh,lr,i)
    (lstm,ytest,prediction,finalLoss) = myTrain(df,seqlen,Nh,lr,30,i)
    Flux.reset!(lstm)
    return finalLoss
end

myObjective (generic function with 1 method)

In [32]:
seqlen=5   #best_params[1]
Nh=5      #best_params[2]
lr=0.002   #best_params[3]
(m,ytest,prediction,finalLoss) = myTrain(df,seqlen,Nh,lr,50,0)
println(finalLoss)

mse_loss(xtest, ytest) = 2.031247f0


┌ Info: Epoch 1
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 2
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 3
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 4
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 5
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 6
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 7
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 8
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 9
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 10
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 11
└ @ Main /Us

(250,)
1.5432965


In [33]:

ho = @hyperopt for i=20,
            sampler = RandomSampler(), # This is default if none provided
            seqlen = StepRange(10, 5, 30),
            Nh = StepRange(2,2, 8),
            lr =  exp10.(LinRange(-3,-2,10))
   println(i,seqlen,Nh,lr)
   @show myObjective(df,seqlen,Nh,lr,i)
end

12540.001291549665014884
mse_loss(xtest, ytest) = 2.3102734f0


┌ Info: Epoch 1
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 2
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 3
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 4
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 5
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 6
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 7
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 8
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 9
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 10
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 11
└ @ Main /Us

(250,)
myObjective(df, seqlen, Nh, lr, i) = 1.71854f0
23040.003593813663804626
mse_loss(xtest, ytest) = 1.9772489f0


┌ Info: Epoch 1
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 2
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 3
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 4
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 5
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 6
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 7
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 8
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 9
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 10
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 11
└ @ Main /Us

(250,)
myObjective(df, seqlen, Nh, lr, i) = 1.6386641f0
32520.01
mse_loss(xtest, ytest) = 1.9825237f0


...

┌ Info: Epoch 1
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 2
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 3
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 4
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 5
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 6
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 7
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 8
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 9
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 10
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 11
└ @ Main /Us

(250,)
myObjective(df, seqlen, Nh, lr, i) = 1.493325f0
203080.0027825594022071257
mse_loss(xtest, ytest) = 1.9953657f0


┌ Info: Epoch 1
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 2
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 3
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 4
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 5
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 6
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 7
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 8
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 9
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 10
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 11
└ @ Main /Us

(250,)
myObjective(df, seqlen, Nh, lr, i) = 1.4941496f0


Hyperoptimizer(20, (:seqlen, :Nh, :lr), (10:5:30, 2:2:8, [0.001, 0.001291549665014884, 0.0016681005372000592, 0.0021544346900318843, 0.0027825594022071257, 0.003593813663804626, 0.0046415888336127815, 0.005994842503189409, 0.007742636826811269, 0.01]), Any[Real[25, 4, 0.001291549665014884], Real[30, 4, 0.003593813663804626], Real[25, 2, 0.01], Real[25, 4, 0.003593813663804626], Real[20, 8, 0.007742636826811269], Real[25, 2, 0.0027825594022071257], Real[25, 8, 0.0016681005372000592], Real[10, 4, 0.0021544346900318843], Real[20, 2, 0.0027825594022071257], Real[30, 6, 0.005994842503189409], Real[25, 8, 0.01], Real[25, 8, 0.0016681005372000592], Real[20, 2, 0.005994842503189409], Real[25, 2, 0.001], Real[20, 2, 0.0027825594022071257], Real[30, 4, 0.01], Real[30, 6, 0.001], Real[10, 6, 0.0021544346900318843], Real[25, 6, 0.007742636826811269], Real[30, 8, 0.0027825594022071257]], Any[1.71854f0, 1.6386641f0, 1.547474f0, 2.1036336f0, 1.7029577f0, 1.6242584f0, 1.5230173f0, 1.7551959f0, 1.73226

In [34]:
best_params, min_f = ho.minimizer, ho.minimum

(Real[30, 6, 0.005994842503189409], 1.4834723f0)

In [35]:
seqlen=best_params[1]
Nh=best_params[2]
lr=best_params[3]
(m,ytest,prediction,finalLoss) = myTrain(df,seqlen,Nh,lr,100,21)
println(finalLoss)


mse_loss(xtest, ytest) = 1.9729358f0


┌ Info: Epoch 1
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 2
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 3
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 4
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 5
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 6
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 7
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 8
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 9
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 10
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 11
└ @ Main /Us

┌ Info: Epoch 85
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 86
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 87
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 88
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 89
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 90
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 91
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 92
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 93
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 94
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 95
└ @

(250,)
1.6863598


In [36]:
sum = 0
N = length(ytest)
for (x,y) in zip(ytest,prediction)
    s = x*y
    if s > 0
        sum += 1
    end
end
println(sum/N)

0.64


In [37]:
#using Pkg; Pkg.generate("ConvGRU")